Here we use a model pretrained on the vanilla dataset and fine-tune it on a cavity dataset with and without noise, and compare its performance with a model trained on these datasets from scratch.

Import external libraries

In [ ]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Load cavity datasets

In [ ]:
X_train = np.load('..\data\X_train_c.npy')  # population in F=1
Y_train = np.load('..\data\Y_train_c.npy')  #normalized to w_L

X_test = np.load('..\data\X_val_C.npy')  # use validation set for testing
Y_test = np.load('..\data\Y_val_C.npy')  # use validation set for testing

Normalize X data

In [ ]:
Level = np.min(X_train)
Height = np.max(X_train) - np.min(X_train)

X_train = (X_train - Level) / Height
X_test = (X_test - Level) / Height

Shuffle training and validation sets

In [ ]:
m_train = np.random.permutation(len(X_train)) # Shuffling mask for X_train
m_test = np.random.permutation(len(X_test)) # Shuffling mask for X_test

X_train, Y_train = X_train[m_train], Y_train[m_train]
X_test, Y_test = X_test[m_test], Y_test[m_test]

Reshape X for CNN

In [ ]:
X_train = X_train[:, :, np.newaxis]
X_test = X_test[:, :, np.newaxis]

Introduce training callbacks

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=500, monitor='val_loss', restore_best_weights=True) # Stop if the validation loss is not improving
learning_rate_cb = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=100) # Decrease the learning rate if the validation loss is not improving
cb_list = [early_stopping_cb, learning_rate_cb]

# Training on cavity data without noise

In [ ]:
model_cnn = keras.models.Sequential([
    keras.layers.Conv1D(kernel_size=16, activation='relu', padding='SAME', filters=32),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Conv1D(kernel_size=8, activation='relu', padding='SAME', filters=64),
    keras.layers.Conv1D(kernel_size=8, activation='relu', padding='SAME', filters=64),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Conv1D(kernel_size=4, activation='relu', padding='SAME', filters=128),
    keras.layers.Conv1D(kernel_size=4, activation='relu', padding='SAME', filters=128),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(2)
])

model_cnn.compile(loss="mae", optimizer=keras.optimizers.Adam(),
                  metrics=["mae"])  #loss functions are given for two output neurons

In [ ]:

pretrained_model = keras.models.load_model('Training_data\model_cnn1.h5')
pretrained_model.compile(loss='mae', optimizer=keras.optimizers.Adam(),
                         metrics=['mae'])  #loss functions are given for two output neurons

history_pretrained = pretrained_model.fit(X_train, Y_train, epochs=2000, validation_data=(X_test, Y_test),
                                          batch_size=32, callbacks=[checkpoint_cb, early_stopping_cb, learning_rate_cb])

history_cnn = model_cnn.fit(X_train, Y_train, epochs=2000, validation_data=(X_test, Y_test), batch_size=32,
                            callbacks=[checkpoint_cb, early_stopping_cb, learning_rate_cb])

In [ ]:
h_cnn = history_cnn.history

cnn_training_data = pd.DataFrame({"CNN training loss": h_cnn["loss"], "CNN test_loss": h_cnn["val_loss"]})
cnn_training_data.index.name = 'CNN Epoch'

fig_cnn = cnn_training_data.plot()
fig_cnn.set_xlabel(r'epoch_count', fontsize=12)
fig_cnn.set_ylabel(r'Loss', fontsize=12)
fig_cnn.set_ylim(0, 0.05)

y = model_cnn.predict(X_test)
print('max error:', np.max(np.abs(y - Y_test)))
print('mean absolute error:', np.mean(np.sqrt((y - Y_test) ** 2)))

In [ ]:
h_pretrained = history_pretrained.history

pretrained_training_data = pd.DataFrame(
    {"Transfer training loss": h_pretrained["loss"], "Transfer test_loss": h_pretrained["val_loss"]})
pretrained_training_data.index.name = 'Transfer Epoch'

fig_pretrained = pretrained_training_data.plot()
fig_pretrained.set_xlabel(r'epoch_count', fontsize=12)
fig_pretrained.set_ylabel(r'Loss', fontsize=12)
fig_pretrained.set_ylim(0, 0.05)

y = pretrained_model.predict(X_test)
print('max error:', np.max(np.abs(y - Y_test)))
print('mean absolute error:', np.mean(np.sqrt((y - Y_test) ** 2)))

# Training with noise

In [ ]:

noise_level = 0.01

noise = np.random.normal(0, noise_level, X_train.shape)
X_train_N = X_train + noise

noise = np.random.normal(0, noise_level, X_test.shape)
X_test_N = X_test + noise

model_cnn = keras.models.Sequential([
    keras.layers.Conv1D(kernel_size=16, activation='relu', padding='SAME', filters=32),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Conv1D(kernel_size=8, activation='relu', padding='SAME', filters=64),
    keras.layers.Conv1D(kernel_size=8, activation='relu', padding='SAME', filters=64),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Conv1D(kernel_size=4, activation='relu', padding='SAME', filters=128),
    keras.layers.Conv1D(kernel_size=4, activation='relu', padding='SAME', filters=128),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(2)
])

model_cnn.compile(loss="mae", optimizer=keras.optimizers.Adam(),
                  metrics=["mae"])  #loss functions are given for two output neurons

history_cnn = model_cnn.fit(X_train_N, Y_train, epochs=2000, validation_data=(X_test_N, Y_test), batch_size=32,
                            callbacks=[checkpoint_cb, early_stopping_cb, learning_rate_cb])

h_cnn = history_cnn.history

cnn_training_data = pd.DataFrame({"CNN training loss": h_cnn["loss"], "CNN test_loss": h_cnn["val_loss"]})
cnn_training_data.index.name = 'CNN Epoch'

fig_cnn = cnn_training_data.plot()
fig_cnn.set_xlabel(r'epoch_count', fontsize=12)
fig_cnn.set_ylabel(r'Loss', fontsize=12)
fig_cnn.set_ylim(0, 0.05)

y = model_cnn.predict(X_test_N)
print('max error:', np.max(np.abs(y - Y_test)))
print('mean absolute error:', np.mean(np.sqrt((y - Y_test) ** 2)))

# #save model
# model_cnn.save('Training_data\with_noise\one_percent\model_cavity_scratch.h5')

# #save data
# cnn_training_data.to_csv('Training_data\with_noise\one_percent\data_cavity_scratch.csv')

pretrained_model_N = keras.models.load_model('Training_data\model_cnn1.h5')
pretrained_model_N.compile(loss='mae', optimizer=keras.optimizers.Adam(),
                           metrics=['mae'])  #loss functions are given for two output neurons

history_pretrained = pretrained_model_N.fit(X_train_N, Y_train, epochs=2000, validation_data=(X_test_N, Y_test),
                                            batch_size=32,
                                            callbacks=[checkpoint_cb, early_stopping_cb, learning_rate_cb])

h_pretrained = history_pretrained.history

pretrained_training_data = pd.DataFrame(
    {"Transfer training loss": h_pretrained["loss"], "Transfer test_loss": h_pretrained["val_loss"]})
pretrained_training_data.index.name = 'Transfer Epoch'

fig_pretrained = pretrained_training_data.plot()
fig_pretrained.set_xlabel(r'epoch_count', fontsize=12)
fig_pretrained.set_ylabel(r'Loss', fontsize=12)
fig_pretrained.set_ylim(0, 0.05)

y = pretrained_model.predict(X_test)
print('max error:', np.max(np.abs(y - Y_test)))
print('mean absolute error:', np.mean(np.sqrt((y - Y_test) ** 2)))

# #save model
# model_cnn.save('Training_data\with_noise\one_percent\model_cavity_finetuned.h5')

# #save data
# cnn_training_data.to_csv('Training_data\with_noise\one_percent\data_cavity_finetuned.csv')


from keras.models import load_model

filter_model = load_model('Training_data\with_noise\one_percent\model_denoise.h5')
tuned_cnn_model = load_model('Training_data\model_cavity_transfer.h5')
y = tuned_cnn_model.predict(X_test_N)
err = np.abs(Y_test - y)
err_df = pd.DataFrame({'W_par': err[:, 0], 'W_ort': err[:, 1]})
err_df.plot()

y_filtered = tuned_cnn_model.predict(filter_model.predict(X_test_N))
err_filtered = np.abs(Y_test - y_filtered)
err_filtered_df = pd.DataFrame({'W_par_filtered': err_filtered[:, 0], 'W_ort_filtered': err_filtered[:, 1]})
err_filtered_df.plot()

X_filtered = filter_model.predict(X_test_N)

index = 150
fig, ax = plt.subplots()
ax.plot(range(320), X_filtered[index, :, 0])
ax.plot(range(320), X_test_N[index, :, 0])
ax.legend(loc=0)